In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from numpy.linalg import norm as norm 
from numpy.linalg import pinv

from sklearn.preprocessing import PolynomialFeatures
from numpy.polynomial import chebyshev as cheb
# cheb.chebvander.__name__ = "Chebyshev"

interval = (-1.0, 1.0)

In [ ]:
X = np.linspace(-1., 1., 1001)
k = 30
nk = 2*(k+1)
M = cheb.chebvander(X, deg=nk)

obs_set = np.arange(k+1)
rand_state = np.random.RandomState(10)

ctrue = np.zeros(nk+1)
ctrue[2*k ] = 1.5
# ctrue[2*k+1] = 1.0
ctrue[1] = 0.5
ctrue += rand_state.randn(ctrue.size)

f = M @ ctrue 

chat_true = np.zeros_like(ctrue)
sol = np.linalg.lstsq(M[:,obs_set], f, rcond=None)
chat_true[obs_set] = sol[0]

inds = np.arange(X.size)
rand_state.shuffle(inds)
Mprime = M[inds]
chat_true2 = np.zeros_like(ctrue)
sol2 = np.linalg.lstsq(Mprime[:,obs_set]/X.size, f[inds]/X.size, rcond=None)
chat_true2[obs_set] = sol[0]

print(np.allclose(chat_true2, chat_true))



print(ctrue)
print(np.round(chat_true, 2))
print(obs_set)

fig, ax = plt.subplots()
ax.plot(X, f, 'k-', label='gt')
ax.plot(X, M@chat_true, label='true estimate')
ax.legend()
plt.show()

In [ ]:
nstart = 5
basis_idx = np.arange(ctrue.size)


def run_simulation(samples, probs=None, nstart=5):
    if probs is None:
        probs = np.ones_like(samples, dtype=float)

    assert probs.size == samples.size

    res = {'chat':[], 'errs':[], 'A_norm':[], 'MTMinv_norm':[], 'MTU_norm':[]}
    T = samples[:nstart]
    U = np.setdiff1d(np.arange(X.size), T) 
    rs = np.random.RandomState(10)
    noise = 0.1*rs.rand(X.size)
    

    for i in range(samples.size - nstart+1):
        MTM = M[np.ix_(T,obs_set)]
        MTU = M[np.ix_(T,np.delete(np.arange(M.shape[1]), obs_set))]
        probsi = np.sqrt(probs[:nstart+i]*(nstart+i))
        #probsi = np.sqrt(probs[:nstart+i])
        y = M[T,:] @ ctrue #+ noise[T]
        sol = np.linalg.lstsq(MTM/probsi.reshape(-1, 1), y/probsi, rcond=None)
        chat = np.zeros_like(ctrue, dtype=float)
        chat[obs_set] = sol[0]
        res['chat'].append(chat) 
        res['errs'].append(np.linalg.norm(M@(ctrue - chat)))
        
        try: 
            Minv = pinv(MTM)
            res['MTMinv_norm'].append(np.log10(norm(Minv)))
            res['MTU_norm'].append(np.log10(norm(MTU)))
            res['A_norm'].append(np.log10(norm(Minv @ MTU)))

        except np.linalg.LinAlgError:
            res['MTMinv_norm'].append(np.nan)
            res['MTU_norm'].append(np.nan)
            res['A_norm'].append(np.nan)

        T = samples[:nstart+i+1]
        U = np.setdiff1d(np.arange(X.size), T)
        
        
        
    return res 


def run_simulation_nodes(ntotal, nstart=5, method='chebyshev'):
    if method == "chebyshev":
        Tx = cheb.chebpts2(nstart)
    elif method == "lattice":
        Tx = np.linspace(-1,1,nstart) 
    else:
        raise NotImplementedError(f"method = {method} not recognized...")

    res = {'chat':[], 'errs':[], 'A_norm':[], 'MTMinv_norm':[], 'MTU_norm':[]}
    

    for i in range(ntotal - nstart):
        M_ = cheb.chebvander(Tx, deg=nk)
        MTM = M_[:,obs_set]
        MTU = M_[:,np.delete(np.arange(M_.shape[1]), obs_set)]
        y = M_ @ ctrue
        sol = np.linalg.lstsq(MTM, y, rcond=None)
        chat = np.zeros_like(ctrue, dtype=float)
        chat[obs_set] = sol[0]
        res['chat'].append(chat) 
        res['errs'].append(np.linalg.norm(M@(chat_true - chat)))
        
        try: 
            Minv = pinv(MTM)
            res['MTMinv_norm'].append(np.log10(norm(Minv)))
            res['MTU_norm'].append(np.log10(norm(MTU)))
            res['A_norm'].append(np.log10(norm(Minv @ MTU)))

        except np.linalg.LinAlgError:
            res['MTMinv_norm'].append(np.nan)
            res['MTU_norm'].append(np.nan)
            res['A_norm'].append(np.nan)
        if method == "chebyshev":
            Tx = cheb.chebpts2(nstart+i+1)
        elif method == "lattice":
            Tx = np.linspace(-1,1,nstart+i+1)
    return res 


In [ ]:
max_samples = 1001
nsims = 3

In [ ]:
import pickle 

with open("res_wgt.pkl", "wb") as f:
    pickle.dump({"rand":RESULTS_RAND, "ls":RESULTS_LS}, f)

In [ ]:
from tqdm import tqdm 

In [ ]:
# Random
RESULTS_RAND = []
for j in tqdm(range(nsims), total=nsims):
    rand_state = np.random.RandomState(45+j)
    samples = np.arange(X.size)
    rand_state.shuffle(samples)
    samples = samples[:max_samples]
    RESULTS_RAND.append(run_simulation(samples, nstart=nstart))


In [ ]:
Qs, _ = np.linalg.qr(M[:,obs_set])
poly_ls_probs_s = np.linalg.norm(Qs, axis=1)**2.
print(poly_ls_probs_s.max(), poly_ls_probs_s.min(), poly_ls_probs_s.sum())
poly_ls_probs_s /= poly_ls_probs_s.sum()

RESULTS_LS = []

for j in tqdm(range(nsims), total=nsims):
    rand_state = np.random.RandomState(45+j)
    poly_ls_samples_s = rand_state.choice(X.size, max_samples, p=poly_ls_probs_s)
    poly_ls_probs_s_on_samples = poly_ls_probs_s[poly_ls_samples_s]
    RESULTS_LS.append(run_simulation(poly_ls_samples_s, poly_ls_probs_s_on_samples, nstart=nstart))


In [ ]:
# Chebyshev nodes
res_cheb = run_simulation_nodes(max_samples, nstart=nstart)


In [ ]:
# lattice nodes
res_lattice = run_simulation_nodes(max_samples, nstart=nstart, method='lattice')


In [ ]:
max_to_plot = 1002 #max_samples
min_to_plot = obs_set.size-2
save = False
key = 'errs'
if key == 'errs':
    ylabel = r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$"
    
elif key == 'A_norm':
    ylabel = r"$\|A\|$"
elif key == 'MTMinv_norm':
    ylabel = r"\|$M_{\mathcal{TM}}^\dagger\|"
else:
    raise NotImplementedError(f"key = {key} not recognized...")

savename = f"{key.lower()}.png"

fig, ax = plt.subplots()
for results, name in zip([RESULTS_RAND, RESULTS_LS], ['Random', 'Lev. Score']):
    errs = np.array([res[key][min_to_plot-nstart:max_to_plot] for res in results])
    errs /= X.size 
    print(errs.shape)
    mean = errs.mean(axis=0)
    std = errs.std(axis=0)
    line = ax.loglog(np.arange(min_to_plot, len(errs[0]) + min_to_plot), mean, label=name)[0]
    # ax.fill_between(np.arange(min_to_plot, len(errs[0])+min_to_plot), mean, mean+std, color=line.get_color(), alpha=0.6)
# cheb_errs = np.array(res_cheb[key][min_to_plot-nstart:max_to_plot])/X.size
# ax.loglog(np.arange(min_to_plot, len(cheb_errs)+min_to_plot), cheb_errs, '--', label='Cheb. Nodes')
# lat_errs = np.array(res_lattice[key][min_to_plot-nstart:max_to_plot])/X.size
# ax.loglog(np.arange(min_to_plot, len(lat_errs)+min_to_plot), lat_errs, '--', label='Lattice Nodes')

ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(ylabel, fontsize=15)
if save:
    plt.savefig(savename, dpi=250, format='png')
plt.show()

In [ ]:
max_to_plot = 700 #max_samples
min_to_plot = nstart + 1 # obs_set.size
save = True
key = 'A_norm'
if key == 'err':
    ylabel = r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$"
    
elif key == 'A_norm':
    ylabel = r"$\|A\|$"
elif key == 'MTMinv_norm':
    ylabel = r"\|$M_{\mathcal{TM}}^\dagger\|"
else:
    raise NotImplementedError(f"key = {key} not recognized...")

savename = f"{key.lower()}.png"

fig, ax = plt.subplots()
for results, name in zip([RESULTS_RAND, RESULTS_LS], ['Random', 'Lev. Score']):
    errs = np.array([res[key][min_to_plot-nstart:max_to_plot] for res in results])
    errs /= X.size 
    mean = errs.mean(axis=0)
    std = errs.std(axis=0)
    line = ax.loglog(np.arange(min_to_plot, len(errs[0]) + min_to_plot), mean, label=name)[0]
    #ax.fill_between(np.arange(nstart, len(errs[0])+nstart), np.maximum(mean-std, 1e-2), mean+std, color=line.get_color(), alpha=0.6)
# cheb_errs = np.array(res_cheb[key][min_to_plot-nstart:max_to_plot])/X.size
# ax.loglog(np.arange(min_to_plot, len(cheb_errs)+min_to_plot), cheb_errs, '--', label='Cheb. Nodes')
# lat_errs = np.array(res_lattice[key][min_to_plot-nstart:max_to_plot])/X.size
# ax.loglog(np.arange(min_to_plot, len(lat_errs)+min_to_plot), lat_errs, '--', label='Lattice Nodes')

ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(ylabel, fontsize=15)
if save:
    plt.savefig(savename, dpi=250, format='png')
plt.show()

In [ ]:
max_to_plot = 700 # max_samples
min_to_plot = nstart + 1 #obs_set.size
save = True
key = 'MTMinv_norm'
if key == 'err':
    ylabel = r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$"
elif key == 'A_norm':
    ylabel = r"$\|A\|$"
elif key == 'MTMinv_norm':
    ylabel = r"$\|M_{\mathcal{TM}}^\dagger\|$"
else:
    raise NotImplementedError(f"key = {key} not recognized...")

savename = f"{key.lower()}.png"

fig, ax = plt.subplots()
for results, name in zip([RESULTS_RAND, RESULTS_LS], ['Random', 'Lev. Score']):
    errs = np.array([res[key][min_to_plot-nstart:max_to_plot] for res in results])
    errs /= X.size 
    mean = errs.mean(axis=0)
    mean[mean < 0] = np.nan
    std = errs.std(axis=0)
    line = ax.loglog(np.arange(min_to_plot, len(errs[0]) + min_to_plot), mean, label=name)[0]
    #ax.fill_between(np.arange(nstart, len(errs[0])+nstart), np.maximum(mean-std, 1e-2), mean+std, color=line.get_color(), alpha=0.6)
# cheb_errs = np.array(res_cheb[key][min_to_plot-nstart:max_to_plot])/X.size
# cheb_errs[cheb_errs < 0] = np.nan
# ax.loglog(np.arange(min_to_plot, len(cheb_errs)+min_to_plot), cheb_errs, '--', label='Cheb. Nodes')
# lat_errs = np.array(res_lattice[key][min_to_plot-nstart:max_to_plot])/X.size
# lat_errs[lat_errs < 0] = np.nan
# ax.loglog(np.arange(min_to_plot, len(lat_errs)+min_to_plot), lat_errs, '--', label='Lattice Nodes')

print(cheb_errs.min())
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(ylabel, fontsize=15)
if save:
    plt.savefig(savename, dpi=250, format='png')
plt.show()

In [ ]:

def plot_results(res, samples, savename, do_anim=True):
    if do_anim:
        ## make gif of the progression of coefficients
        niter = len(res['chat'])
        fig, ax = plt.subplots()
        ax.scatter(basis_idx, ctrue, marker='o', s=20, c='k')
        color = iter(plt.cm.viridis(np.linspace(0, 1, niter)))
        scat = ax.scatter(basis_idx, res['chat'][0], label=0, marker='x')
        ax.set_title("Iter = 0")
        ax.set_ylim(ctrue.min()-0.5, ctrue.max()+0.5)

        def animate(i):
            scat.set_offsets(np.hstack((basis_idx.reshape(-1,1), res['chat'][i].reshape(-1,1))))
            # scat.set_color(basis_idx.size*[c])
            ax.set_title(f"Iter = {i+1}")
            return scat, 

        ani = animation.FuncAnimation(fig, animate, repeat=True,
                                            frames=len(res['chat']) - 1, interval=50)
        # To save the animation using Pillow as a gif
        writer = animation.PillowWriter(fps=5,
                                        metadata=dict(artist='Me'),
                                        bitrate=1800)
        ani.save(f'scatter_{savename}.gif', writer=writer)

        plt.show()



    ## plot estimated functions
    niter = 10
    itermax = 20
    fig, ax = plt.subplots()
    ax.plot(X, f, 'k-', alpha=0.7)
    ax.scatter(X[samples[:nstart+itermax]], M[samples[:nstart+itermax]] @ ctrue, marker='o', s=20, c='k')
    color = iter(plt.cm.viridis(np.linspace(0, 1, niter)))
    for i in range(itermax-niter,itermax):
        c = next(color)
        Ti = samples[:nstart+i]
        Design_mat = M[np.ix_(Ti, obs_set)]
        ax.scatter(X[Ti], Design_mat @ res['chat'][i][obs_set], label=i, marker='x', c=Ti.size*[c])
        ax.plot(X, M @ res['chat'][i], c=c)

    ax.legend(title='Iteration', bbox_to_anchor=(1.2, 1.05))
    plt.show()


    if do_anim:
        ## make gif of the progression of coefficients
        niter = len(res['chat'])
        fig, ax = plt.subplots()
        ax.plot(X, f, 'k-', alpha=0.7)
        ax.plot(X, M@chat_true, 'r-', alpha=0.8)
        #ax.scatter(X[T[:nstart]], M[T[:nstart]] @ ctrue, marker='o', s=20, c='k')
        Ti = samples[:nstart]
        Design_mat = M[np.ix_(Ti, obs_set)]
        scat = ax.scatter(X[Ti], Design_mat @ res['chat'][0][obs_set], marker='x')
        line = ax.plot(X, M @ res['chat'][0])[0]
        ax.set_title("Iter = 0")
        ax.set_ylim(f.min()-0.1, f.max()+0.1)

        def animate(i):
            Ti = samples[:nstart+i]
            Design_mat = M[np.ix_(Ti, obs_set)]
            yscat = Design_mat @ res['chat'][i][obs_set]
            scat.set_offsets(np.hstack((X[Ti].reshape(-1,1), yscat.reshape(-1,1))))
            line.set_ydata(M @ res['chat'][i])
            ax.set_title(f"Iter = {i+1}")
            return (scat, line)

        ani = animation.FuncAnimation(fig, animate, repeat=True,
                                            frames=len(res['chat']) - 1, interval=50)
        # To save the animation using Pillow as a gif
        writer = animation.PillowWriter(fps=5,
                                        metadata=dict(artist='Me'),
                                        bitrate=1800)
        ani.save(f'plot_{savename}.gif', writer=writer)

        plt.show()

In [ ]:
Q, _ = np.linalg.qr(M)
poly_ls_probs = np.linalg.norm(Q, axis=1)**2.
print(poly_ls_probs.max(), poly_ls_probs.min(), poly_ls_probs.sum())
poly_ls_probs /= poly_ls_probs.sum()

rand_state = np.random.RandomState(45)

poly_ls_samples = rand_state.choice(X.size, max_samples, p=poly_ls_probs)
print(poly_ls_samples.size)
poly_ls_probs_on_samples = poly_ls_probs[poly_ls_samples]

# fig, ax = plt.subplots()
# ax.scatter(X, poly_ls_probs)
# plt.show()

In [ ]:
res_ls_poly = run_simulation(poly_ls_samples, poly_ls_probs_on_samples, nstart=nstart)

#plot_results(res_ls_poly, poly_ls_samples, 'ls_poly')

In [ ]:
# plot_results(res_ls_poly, poly_ls_samples, 'ls_poly')

In [ ]:
Qs, _ = np.linalg.qr(M[:,obs_set])
poly_ls_probs_s = np.linalg.norm(Qs, axis=1)**2.
print(poly_ls_probs_s.max(), poly_ls_probs_s.min(), poly_ls_probs_s.sum())
poly_ls_probs_s /= poly_ls_probs_s.sum()

rand_state = np.random.RandomState(45)

poly_ls_samples_s = rand_state.choice(X.size, max_samples, p=poly_ls_probs_s)
print(poly_ls_samples_s.size)
poly_ls_probs_s_on_samples = poly_ls_probs_s[poly_ls_samples_s]

# fig, ax = plt.subplots()
# ax.scatter(X, poly_ls_probs_s)
# plt.show()

res_ls_poly_s = run_simulation(poly_ls_samples_s, poly_ls_probs_s_on_samples, nstart=nstart)

#plot_results(res_ls_poly_s, poly_ls_samples_s, 'ls_poly2')

In [ ]:

# plot_results(res_cheb, poly_ls_samples_s, 'ls_poly2')

In [ ]:
import matplotlib 
plt.rcParams.setdefault('font.family')
plt.rcParams.update({
    "text.usetex": True,
    # "font.family": "sans-serif"
})

In [ ]:
max_to_plot = max_samples
fig, ax = plt.subplots()
for results, name in zip([RESULTS_RAND, RESULTS_LS], ['Random', 'Lev. Score']):
    errs = np.array([res['errs'][:max_to_plot] for res in results])
    errs /= X.size #(nstart +np.arange(len(errs), dtype=float))
    print(errs.shape)
    ax.loglog(np.arange(nstart, len(errs[0])+nstart), errs.mean(axis=0), label=name)
cheb_errs = res_cheb['errs'][:max_to_plot]
ax.loglog(np.arange(nstart, len(cheb_errs)+nstart), cheb_errs, label='Cheb. Nodes')
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$", fontsize=15)
# plt.savefig("ls_vs_rand_poly.png", dpi=250, format='png')
plt.show()


In [ ]:
max_to_plot = max_samples
fig, ax = plt.subplots()
for res, name in zip([res_rand, res_ls_poly_s, res_cheb], ['Random', 'Lev. Score', 'Cheb. Nodes']): 
    errs = np.array(res['errs'][:max_to_plot])
    errs /= X.size #(nstart +np.arange(len(errs), dtype=float))
    ax.loglog(np.arange(nstart, len(errs)+nstart), errs, label=name)
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(r"Error, $\|\hat{f}_{\mathcal{T}} - f\|_2$", fontsize=15)
plt.savefig("ls_vs_rand_poly.png", dpi=250, format='png')
plt.show()

fig, ax = plt.subplots()
for res, name in zip([res_rand, res_ls_poly_s, res_cheb], ['Random', 'Lev. Score', 'Cheb. Nodes']): 
    chats = res['chat'][:max_to_plot]
    errs = np.array([np.linalg.norm(ctrue - chat) for chat in chats])
    ax.loglog(np.arange(nstart, len(errs)+nstart), errs, label=name)
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(r"Error, $\|\hat{\theta}_{\mathcal{T}} - \theta\|_2$", fontsize=15)
plt.savefig("ls_vs_rand_poly_theta.png", dpi=250, format='png')
plt.show()

In [ ]:
print(res_cheb['chat'][-1][obs_set])
print(chat_true[obs_set])
print(res_rand['chat'][-1][obs_set])
print(np.allclose(X, X[np.sort(samples)]))

In [ ]:
M = cheb.chebvander(X, deg=nk)
fig, ax = plt.subplots()
ax.plot(X, f, 'k-', label='gt')
ax.plot(X, M@res_cheb['chat'][-1], 'r-', label='cheb')
ax.plot(X, M@chat_true, 'g-', label='full')
ax.legend()
plt.show()

In [ ]:
max_to_plot = 400
fig, ax = plt.subplots()
for res, name in zip([res_rand, res_ls_poly, res_ls_poly_s, res_cheb], ['Random', 'Lev. Score', 'Lev. Score Sub', 'Cheb. Nodes']): 
    errs = res['A_norm'][:max_to_plot]
    errs /= (nstart +np.arange(len(errs), dtype=float))
    ax.loglog(np.arange(nstart, len(errs)+nstart), errs, label=name)
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(r"$\|A\|$", fontsize=15)
# plt.savefig("ls_vs_rand_poly.png", dpi=250, format='png')
plt.show()

fig, ax = plt.subplots()
for res, name in zip([res_rand, res_ls_poly, res_ls_poly_s, res_cheb], ['Random', 'Lev. Score', 'Lev. Score Sub', 'Cheb. Nodes']): 
    errs = res['MTMinv_norm'][:max_to_plot]
    errs /= (nstart +np.arange(len(errs), dtype=float))
    ax.loglog(np.arange(nstart, len(errs)+nstart), errs, label=name)
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(r"$\|M_{\mathcal{TM}}^\dagger\|$", fontsize=15)
# plt.savefig("ls_vs_rand_poly.png", dpi=250, format='png')
plt.show()

fig, ax = plt.subplots()
for res, name in zip([res_rand, res_ls_poly, res_ls_poly_s, res_cheb], ['Random', 'Lev. Score', 'Lev. Score Sub', 'Cheb. Nodes']): 
    errs = res['MTU_norm'][:max_to_plot]
    errs /= (nstart +np.arange(len(errs), dtype=float))
    ax.loglog(np.arange(nstart, len(errs)+nstart), errs, label=name)
ax.legend(title='Sampling Method', fontsize=11)
ax.set_xlabel(r"Size of Training Set, $\mathcal{T}$", fontsize=15)
ax.set_ylabel(r"$\|M_{\mathcal{TU}}\|$", fontsize=15)
# plt.savefig("ls_vs_rand_poly.png", dpi=250, format='png')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, f, 'k-', label=r'ground truth, $f$')
ax.plot(X, M@chat_true, 'g-', label=r'"full" estimate, $\hat{f}$')
line = ax.plot(X, M@res_ls_poly['chat'][80], 'r-', label=r'estimate, $\hat{f}_{\mathcal{T}}$')[0]
#ax.scatter(X[poly_ls_samples[:nstart +20]], f[poly_ls_samples[:nstart +20]], marker='x', c=line.get_color(), zorder=10)
ax.legend(fontsize=13)
ax.set_xlabel(r'$x$', fontsize=15)
ax.set_ylabel(r'$f(x)$', fontsize=15)
plt.savefig('gt_vs_full.png', dpi=250, format='png')
plt.show()

# Observations

Why doesn't this do as well as we would think that leverage score sampling would? Is it because of the basis? Maybe try doing polynomials, and make sure this regression framework code works over there. 


Ideas:
* Higher dimensional example to see the benefit of leverage score sampling?
* Investigate how using Chebyshev nodes helps in this setting, does it essentially do like random or leverage score sampling?
* Perhaps what's more useful is looking at the norms of the matrices, not the convergence to the "full estimate".

In [ ]:
pts = cheb.chebpts2(100)
print(pts)

fig, ax = plt.subplots()
ax.scatter(pts, np.ones_like(pts))
plt.show()